# DataFrame

A DataFrame represents a tabular, spreadsheet-like data structure containing an ordered collection of columns, each of which can be a different value type (numeric, string, boolean, etc.)

There are numerous ways to make a DataFrame. One of the most common ways is from a dictionary of equal-length lists or NumPy arrays.

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

data = {"state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada"],
        "year": [2000, 2001, 2002, 2001, 2002],
        "pop": [1.5, 1.7, 3.6, 2.4, 2.9]}
frame = DataFrame(data)
frame

,pop,state,year
0,1.5,Ohio,2000
1,1.7,Ohio,2001
2,3.6,Ohio,2002
3,2.4,Nevada,2001
4,2.9,Nevada,2002


You can also add specify the order you want for your columns (notice before they aren't automatically in the order they are listed in the dictionary).

In [2]:
DataFrame(data, columns = ["year", "state", "pop"])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9


If you pass a column that isn't contained in DataFrame, it will appear with NaN (not a number) values in the result.

In [3]:
frame2 = DataFrame(data, columns = ["year", "state", "pop", "debt"], index = ["one", "two", "three", "four", "five"])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN


Use .columns for a list of the columns in a particular DataFrame.

In [4]:
frame2.columns

Index([u'year', u'state', u'pop', u'debt'], dtype='object')

A column in a DataFrame can be retrieved as a Series either by dictionary-like notation or by attribute.

In [5]:
frame2["state"]

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
Name: state, dtype: object

In [6]:
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
Name: year, dtype: int64

Rows can also be retrieved by position or name through the ix indexing field method (more on this later).

In [7]:
frame2.ix["three"]

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

Columns can be modified by assignment. For example, the empty "debt" column could be assigned a scalar value or an array of values.

In [8]:
frame2["debt"] = 16.5
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5


When assigning lists or arrays to a column, the value's length must match the length of the DataFrame. If you assign a Series, it will instead be conformed exactly to the DataFrame's index, inserting missing values (NaN) into any holes.

In [9]:
val = Series([-1.2, -1.5, -1.7], index = ["two", "four", "five"])
frame2["debt"] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7


Assigning a column that doesn't exist creates a new column. The del keyword deletes columns as with a dictionary.

In [10]:
frame2["eastern"] = frame2.state == "Ohio"
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False


In [11]:
del frame2["eastern"]
frame2.columns

Index([u'year', u'state', u'pop', u'debt'], dtype='object')

Another common form of data is a nested dictionary of dictionaries format. If passed into DataFrame, it will interpret the outer keys as the columns and the inner keys as the rows.

In [12]:
pop = {"Nevada": {2001:2.4, 2002:2.9}, "Ohio": {2000:1.5, 2001: 1.7, 2002: 3.6}}
frame3 = DataFrame(pop)
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


You can then transpose the result.

In [13]:
frame3.T

,2000,2001,2002
Nevada,NaN,2.4,2.9
Ohio,1.5,1.7,3.6


You can also choose to set explicit index values.

In [14]:
DataFrame(pop, index=[2001,2002,2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


If a DataFrame's index and columns have their name attributes set, these will also be displayed.

In [15]:
frame3.index.name = "year"; frame3.columns.name = "state"
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


Like Series, the values attribute returns the data contained in the DataFrame as a 2D ndarray.

In [16]:
frame3.values

array([[ nan,  1.5],
       [ 2.4,  1.7],
       [ 2.9,  3.6]])

If the DataFrame's columns are different dtypes, the dtype of the values array will be chosen to accomodate all of the columns.

In [17]:
frame2.values

array([[2000L, 'Ohio', 1.5, nan],
       [2001L, 'Ohio', 1.7, -1.2],
       [2002L, 'Ohio', 3.6, nan],
       [2001L, 'Nevada', 2.4, -1.5],
       [2002L, 'Nevada', 2.9, -1.7]], dtype=object)

### Index Objects

Index objects in pandas are responsible for holding the axis labels and other metadata (like the axis name or names). Any array or other sequence of labels used when constructing a Series or DataFrame is internally converted to an Index.

In [18]:
obj = Series(range(3), index = ["a", "b", "c"])
index = obj.index
index

Index([u'a', u'b', u'c'], dtype='object')

In [19]:
index[1:]

Index([u'b', u'c'], dtype='object')

Note that Index objects are immutable and thus can't be modified by the user. Immutability is important so that Index objects can be safely shared among data structures. index[1] = "d" will result in an error because index is an Index object, therefore it can't be changed.

In [20]:
index = pd.Index(np.arange(3))
obj2 = Series([1.5, -2.5, 0], index = index)
obj2.index is index

True

In addition to being array-like, and Index also functions as a fixed-size set.

In [21]:
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [22]:
"Ohio" in frame3.columns

True

In [23]:
2003 in frame3.index

False

### Reindexing

A critical method on pandas objects is reindex, which means to create a new object with the data conformed to a new index.

In [24]:
obj = Series([4.5,7.2,-5.3,3.6], index = ["d", "b", "a", "c" ])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

Calling reindex on this Series rearranges the data according to the new index, introducing missing values if any index values were not already present.

In [25]:
obj2 = obj.reindex(["a", "b", "c", "d", "e"])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [26]:
obj.reindex(["a", "b", "c", "d", "e"], fill_value = 0)

a   -5.3
b    7.2
c    3.6
d    4.5
e    0.0
dtype: float64

For ordered data like time series, it may be desirable to do some interpolation or filling of values when reindexing. The method option allows us to do this, using a method such as ffill which forward fills the values (bfill would do the same thing except backwards).

In [27]:
obj3 = Series(["blue", "purple", "yellow"], index = [0,2,4])
obj3.reindex(range(6), method = "ffill")

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

With DataFrame, reindex can alter either the (row) index, columns or both. When passed in just a sequence, the rows are reindexed in the result.

In [28]:
frame = DataFrame(np.arange(9).reshape((3,3)), index = ["a", "c", "d"], columns = ["Ohio", "Texas", "California"])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [29]:
frame2 = frame.reindex(["a", "b", "c", "d"])
frame2

,Ohio,Texas,California
a,0,1,2
b,NaN,NaN,NaN
c,3,4,5
d,6,7,8


The columns can be reindexed using the columns keyword.

In [30]:
states = ["Texas", "Utah", "California"]
frame.reindex(columns = states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


Both can be reindexed in one shot, though interpolation will only apply row-wise (axis 0).

In [31]:
frame.reindex(index = ["a", "b", "c", "d"], method = "ffill", columns = states)

,Texas,Utah,California
a,1,NaN,2
b,1,NaN,2
c,4,NaN,5
d,7,NaN,8


Reindexing can be done more succinctly by label-indexing with ix.

In [32]:
frame.ix[["a", "b", "c", "d"], states]

,Texas,Utah,California
a,1,NaN,2
b,NaN,NaN,NaN
c,4,NaN,5
d,7,NaN,8


### Dropping entries from an axis

Dropping one or more values from an axis is easy if you have an index array or list without those entries. The drop method will return a new object with the indicated value or values deleted from an axis.

In [33]:
obj = Series(np.arange(5.), index = ["a", "b", "c", "d", "e"])
new_obj = obj.drop("c")
new_obj

a    0
b    1
d    3
e    4
dtype: float64

In [34]:
obj.drop(["d", "c"])

a    0
b    1
e    4
dtype: float64

Index values can be deleted from either axis in a DataFrame.

In [35]:
data = DataFrame(np.arange(16).reshape((4,4)), index = ["Ohio", "Colorado", "Utah", "New York"], columns = ["one", "two", "three", "four"])
data.drop(["Colorado", "Ohio"])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [36]:
data.drop("two", axis = 1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [37]:
data.drop(["two", "four"], axis = 1)

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


### Indexing, Selection, and filtering

Series indexing (obj[...]) works analogously to NumPy array indexing, except you can use the Series's index values instead of only integers.

In [38]:
obj = Series(np.arange(4.), index = ["a", "b", "c", "d"])
obj["b"]

1.0

In [39]:
obj[1]

1.0

In [40]:
obj[2:4]

c    2
d    3
dtype: float64

In [41]:
obj[["b", "a", "d"]]

b    1
a    0
d    3
dtype: float64

In [42]:
obj[[1,3]]

b    1
d    3
dtype: float64

In [43]:
obj[obj < 2]

a    0
b    1
dtype: float64

Slicing with labels behaves differently than normal Python slicing in that the endpoint is inclusive.

In [44]:
obj["b":"c"]

b    1
c    2
dtype: float64

Setting using these methods works as well.

In [45]:
obj["b":"c"] = 5
obj

a    0
b    5
c    5
d    3
dtype: float64

As seen before, indexing into a DataFrame is for retrieving one or more columns either with a single value or sequence.

In [46]:
data = DataFrame(np.arange(16).reshape((4,4)), index = ["Ohio", "Colorado", "Utah", "New York"], columns = ["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [47]:
data["two"]

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int32

In [48]:
data[["three", "one"]]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


Indexing like this has a few special cases. First selecting rows by slicing or a boolean array.

In [49]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [50]:
data[data["three"] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


Another use case is in indexing with a boolean DataFrame, such as one produced by a scalar comparison.

In [51]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [52]:
data[data < 5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


The special indexing field ix enables you to select a subset of the rows and columns from a DataFrame with NumPy-like notation plus axis labels. This can be considered another way to do reindexing.

In [53]:
data.ix["Colorado", ["two", "three"]]

two      5
three    6
Name: Colorado, dtype: int32

In [54]:
data.ix[["Colorado", "Utah"], [3,0,1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


In [55]:
data.ix[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

In [56]:
data.ix[:"Utah", "two"]

Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int32

In [57]:
data.ix[data.three > 5, :3]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


### Arithmetic and Data Alignment

One of the most important pandas features is the behavior of arithmetic between objects with different indexes. When adding objects together, if any pairs are not the same, the respective index in the result will be the union of the index pairs.

The internal data alignment introduces NaN values in the indices that don't overlap. Missing values propogate in arithmetic computations.

In [58]:
s1 = Series([7.3, -2.5, 3.4, 1.5], index = ["a", "c", "d", "e"])
s2 = Series([-2.1, 3.6, -1.5, 4, 3.1], index = ["a", "c", "e", "f", "g"])

In [59]:
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [60]:
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [61]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

In the case of the DataFrame, alignment is performed on both the rows and the columns.

In [62]:
df1 = DataFrame(np.arange(9.).reshape((3,3)), columns = list("bcd"), index = ["Ohio", "Texas", "Colorado"])
df2 = DataFrame(np.arange(12.).reshape((4,3)), columns = list("bde"), index = ["Utah", "Ohio", "Texas", "Oregon"])

In [63]:
df1

,b,c,d
Ohio,0,1,2
Texas,3,4,5
Colorado,6,7,8


In [64]:
df2

,b,d,e
Utah,0,1,2
Ohio,3,4,5
Texas,6,7,8
Oregon,9,10,11


Adding these together returns a DataFrame whose index and columns are the unions of the ones in each DataFrame.

In [65]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3,NaN,6,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9,NaN,12,NaN
Utah,NaN,NaN,NaN,NaN


#### Arithmetic methods with fill values

If you don't want your DataFrame to be full of null values, you have it filled in with a special value (like 0) when the axis label is found in one object but not the other.

In [66]:
df1 = DataFrame(np.arange(12.).reshape((3, 4)), columns = list("abcd"))
df2 = DataFrame(np.arange(20.).reshape((4, 5)), columns = list("abcde"))

In [67]:
df1

,a,b,c,d
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [68]:
df2

,a,b,c,d,e
0,0,1,2,3,4
1,5,6,7,8,9
2,10,11,12,13,14
3,15,16,17,18,19


In [69]:
df1 + df2

,a,b,c,d,e
0,0,2,4,6,NaN
1,9,11,13,15,NaN
2,18,20,22,24,NaN
3,NaN,NaN,NaN,NaN,NaN


Using the add method of df1, we can pass df2 and an arrangement to fill_value.

In [70]:
df1.add(df2, fill_value = 0)

,a,b,c,d,e
0,0,2,4,6,4
1,9,11,13,15,9
2,18,20,22,24,14
3,15,16,17,18,19


Similarly, when reindexing a Series or DataFrame, you can also specify a different value.

In [71]:
df1.reindex(columns = df2.columns, fill_value = 0)

,a,b,c,d,e
0,0,1,2,3,0
1,4,5,6,7,0
2,8,9,10,11,0


#### Operations between DataFrame and Series

As with NumPy arrays, arithmetic between DataFrame and Series is well defined. Consider the difference between a 2D array and one of its rows.

In [72]:
arr = np.arange(12.).reshape((3, 4))
arr

array([[  0.,   1.,   2.,   3.],
       [  4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.]])

In [73]:
arr[0]

array([ 0.,  1.,  2.,  3.])

In [74]:
arr - arr[0]

array([[ 0.,  0.,  0.,  0.],
       [ 4.,  4.,  4.,  4.],
       [ 8.,  8.,  8.,  8.]])

This is referred to as _broadcasting_. Operations between a DataFrame and a Series are similar.

In [75]:
frame = DataFrame(np.arange(12.).reshape((4, 3)), columns = list("bde"), index = ["Utah", "Ohio", "Texas", "Oregon"])
series = frame.ix[0]
frame

,b,d,e
Utah,0,1,2
Ohio,3,4,5
Texas,6,7,8
Oregon,9,10,11


In [76]:
series

b    0
d    1
e    2
Name: Utah, dtype: float64

By default, arithmetic between DataFrame and Series matches the index of the Series on the DataFrame's columns, broadcasting down the rows.

In [77]:
frame - series

,b,d,e
Utah,0,0,0
Ohio,3,3,3
Texas,6,6,6
Oregon,9,9,9


If an index isn't found in either the DataFrame's columns or the Series's index, the objects will be reindexed to form the union.

In [78]:
series2 = Series(range(3), index = ["b", "e", "f"])
frame + series2

,b,d,e,f
Utah,0,NaN,3,NaN
Ohio,3,NaN,6,NaN
Texas,6,NaN,9,NaN
Oregon,9,NaN,12,NaN


If you would rather broadcast over the columns, matching on the rows, you have to use one of the arithmetic methods.

Here is a reminder on some arithmetic methods:
* add (method for addition)
* sub (method for subtraction)
* div (method for division)
* mul (method for multiplication)

In [79]:
series3 = frame["d"]

In [80]:
frame

,b,d,e
Utah,0,1,2
Ohio,3,4,5
Texas,6,7,8
Oregon,9,10,11


In [81]:
series3

Utah       1
Ohio       4
Texas      7
Oregon    10
Name: d, dtype: float64

The axis number that you pass is the _axis to match on_. In this case, we mean to match on the DataFrame's row index and broadcast across.

In [82]:
frame.sub(series3, axis = 0)

,b,d,e
Utah,-1,0,1
Ohio,-1,0,1
Texas,-1,0,1
Oregon,-1,0,1


### Sorting and Ranking

Sorting a data set by some criterion is another important built-in operation. To sort lexicographically by row or column index, use the sort_index method, which returns a new, sorted object.

In [83]:
obj = Series(range(4), index = ["d", "a", "b", "c"])
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

With a DataFrame, you can sort by index on either axis.

In [84]:
frame = DataFrame(np.arange(8).reshape((2,4)), index = ["three", "one"], columns = ["d", "a", "b", "c"])
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [85]:
frame.sort_index(axis = 1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


The data is sorted in ascending order by default, but can be sorted in descending order too.

In [86]:
frame.sort_index(axis = 1, ascending = False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


To sort a Series by its values, use its order method.

In [87]:
obj = Series([4, 7, -3, 2])
obj.order()

2   -3
3    2
0    4
1    7
dtype: int64

Any missing values are sorted to the end of the Series by default.

In [88]:
obj = Series([4, np.nan, 7, np.nan, -3, 2])
obj.order()

4    -3
5     2
0     4
2     7
1   NaN
3   NaN
dtype: float64

On DataFrame, you may want to sort by the values in one or more columns. To do so, pass one or more column names to the by option.

In [89]:
frame = DataFrame({"b": [4, 7, -3, 2], "a": [0, 1, 0, 1]})
frame

,a,b
0,0,4
1,1,7
2,0,-3
3,1,2


In [90]:
frame.sort_index(by = "b")

,a,b
2,0,-3
3,1,2
0,0,4
1,1,7


To sort by multiple columns, pass a list of names.

In [91]:
frame.sort_index(by = ["a", "b"])

,a,b
2,0,-3
0,0,4
3,1,2
1,1,7


_Ranking_ is closely related to sorting, assigning ranks from one through the number of valid data points in an array. It is similar to the indirect sort indeces produced by numpy.argsort, except that ties are broken according to a rule. By default rank breaks ties by assigning each group the mean rank.

In [92]:
obj = Series([7, -5, 7, 4, 2, 0, 4])
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

Ranks can also be assigned according to the order they're observed in the data.

In [93]:
obj.rank(method = "first")

0    6
1    1
2    7
3    4
4    3
5    2
6    5
dtype: float64

You can rank in descending order too.

In [94]:
obj.rank(ascending = False, method = "max")

0    2
1    7
2    2
3    4
4    5
5    6
6    4
dtype: float64

DataFrame can compute ranks over rows or the columns.

In [95]:
frame = DataFrame({"b": [4.3, 7, -3, 2], "a": [0, 1, 0, 1], "c": [-2, 5, 8, -2.5]})
frame

,a,b,c
0,0,4.3,-2.0
1,1,7.0,5.0
2,0,-3.0,8.0
3,1,2.0,-2.5


In [96]:
frame.rank(axis = 1)

,a,b,c
0,2,3,1
1,1,3,2
2,2,1,3
3,2,3,1


### Axis indexes with duplicate values

So far, all the of the examples have has unique axis labels (index values). Many pandas functions (like pandas) require that labels are unique, but it's not mandatory. Consider a small Series with duplicate indices.

In [97]:
obj = Series(range(5), index = ["a", "a", "b", "b", "c"])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

The index's is_unique property can tell you whether its values are unique or not.

In [98]:
obj.index.is_unique

False

Data selection is one of the main things that behaves differently with duplicates. Indexing a value with multiple entries returns a Series while single entries return a scalar value.

In [99]:
obj["a"]

a    0
a    1
dtype: int64

In [100]:
obj["c"]

4

The same logic extends to indexing rows in a DataFrame.

In [101]:
df = DataFrame(np.random.randn(4,3), index = ["a", "a", "b", "b"])
df

,0,1,2
a,-0.588784,-0.887445,0.034336
a,-0.578913,-1.550616,1.191152
b,0.250803,1.024215,0.112610
b,1.299269,-0.082571,-0.521498


In [102]:
df.ix["b"]

,0,1,2
b,0.250803,1.024215,0.112610
b,1.299269,-0.082571,-0.521498


## Summarizing and Computing Descriptive Statistics

Pandas objects are equipped with a set of common mathematical and statistical methods. Most of these fall into the category of _reductions_ or _summary statistics_, methods that extract a single value (like a sum or mean) from a Series or a Series of values from the rows or columns of a DataFrame. Compared with the equivalent methods of NumPy arrays, they are built to exclude missing data.

In [103]:
df = DataFrame([[1.4, np.nan], [7.1, -4.5], [np.nan, np.nan], [.75, -1.3]], index = ["a", "b", "c", "d"], columns = ["one", "two"])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


Calling DataFrame's sum method returns a Series containing column sums.

In [104]:
df.sum()

one    9.25
two   -5.80
dtype: float64

Passing axis=1 sums over the rows instead.

In [105]:
df.sum(axis = 1)

a    1.40
b    2.60
c     NaN
d   -0.55
dtype: float64

NA values are excluded unless the entire slice (row or column in this case) is NA. This can be disabled using the skipna option.

In [106]:
df.mean(axis = 1, skipna = False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

In [107]:
df.mean(axis = 1, skipna = True)

a    1.400
b    1.300
c      NaN
d   -0.275
dtype: float64

Some methods, like idxmin and idxmax return indirect statistics like the index value where the minimum or maximum values are attained.

In [108]:
df.idxmax()

one    b
two    d
dtype: object

In [109]:
df.idxmin()

one    d
two    b
dtype: object

Other methods are _accumulations_ (they add each new term as you go down the indexes.

In [110]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


Another method that is neither a reduction nor an accumulation is describe, which produces multiple summary statistics in one shot.

In [111]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


On numeric data, describe produces alternate summary statistics.

In [112]:
obj = Series(["a", "a", "b", "c"] * 4)
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object